In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
%pip install google-cloud-bigquery
%pip install pandas
%pip install db-dtypes
%pip install pandas-gbq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from google.cloud import bigquery
import pandas as pd

from utils.import_table import import_table
from utils.export import export_table


In [4]:
# Initialiser le client BigQuery
client = bigquery.Client.from_service_account_json('../../key.json')

In [ ]:
date = import_table(client, 'date', dataset_id='warehouse')
display(date.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,date
0,1,1970-01-08
1,2,1970-01-22
2,3,1970-02-15
3,4,1970-02-24
4,5,1970-03-06


In [ ]:
so = import_table(client, 'sales_order', dataset_id="ensemble_de_donnees_public").sort_values(by='id')

# replace date in so by date_id of date 
so = so.merge(date, on='date', how='left')
so = so.drop(columns=['date'])
so = so.rename(columns={'id_y': 'date_id', 'id_x': 'id'})
display(so.head())


/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,customer_id,store_id,date_id
0,1,119,9,194
1,2,12,5,196
2,3,255,5,786
3,4,65,3,585
4,5,52,8,305


In [ ]:
sol = import_table(client, 'sales_order_line', dataset_id="ensemble_de_donnees_public").sort_values(by='sales_order_id')

sol = sol.merge(so, left_on='sales_order_id', right_on='id', how='left')

sol = sol.drop(columns=['id_y'])
sol = sol.rename(columns={'id_x': 'id'})

display(sol.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,quantity,unit_price,sales_order_id,product_id,customer_id,store_id,date_id
0,1108,1,65,1,123,119,9,194
1,739,3,50,1,58,119,9,194
2,747,3,10,1,130,119,9,194
3,123,2,55,1,139,119,9,194
4,588,3,52,1,19,119,9,194


In [ ]:
stores = import_table(client, 'store', dataset_id="ensemble_de_donnees_public")
stores = stores[['id', 'location_id']]
display(stores.head())

sol = sol.merge(stores, left_on='store_id', right_on='id', how='left')
sol = sol.drop(columns=['id_y', 'store_id'])
sol = sol.rename(columns={'id_x': 'id'})

display(sol.head())

/Users/antoine/miniconda3/envs/AI/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,id,location_id
0,1,1
1,2,77
2,3,35
3,4,22
4,5,6


,id,quantity,unit_price,sales_order_id,product_id,customer_id,date_id,location_id
0,1108,1,65,1,123,119,194,74
1,739,3,50,1,58,119,194,74
2,747,3,10,1,130,119,194,74
3,123,2,55,1,139,119,194,74
4,588,3,52,1,19,119,194,74


In [9]:
sol['total_line'] = sol['quantity'] * sol['unit_price']
sol['total_order'] = sol.groupby('sales_order_id')['total_line'].transform('sum')

display(sol.head())

,id,quantity,unit_price,sales_order_id,product_id,customer_id,date_id,location_id,total_line,total_order
0,1108,1,65,1,123,119,194,74,65,633
1,739,3,50,1,58,119,194,74,150,633
2,747,3,10,1,130,119,194,74,30,633
3,123,2,55,1,139,119,194,74,110,633
4,588,3,52,1,19,119,194,74,156,633


In [10]:
sol = sol.drop(columns='id')
display(sol.head())

,quantity,unit_price,sales_order_id,product_id,customer_id,date_id,location_id,total_line,total_order
0,1,65,1,123,119,194,74,65,633
1,3,50,1,58,119,194,74,150,633
2,3,10,1,130,119,194,74,30,633
3,2,55,1,139,119,194,74,110,633
4,3,52,1,19,119,194,74,156,633


In [ ]:
# Variables pour BigQuery
dataset_name = "warehouse" # Nom du dataset à créer/exporter
table_name = "f_sale" # Nom de la table à créer


In [13]:
export_table(client, sol, dataset_name, table_name)

Dataset warehouse_test déjà existant.


100%|██████████| 1/1 [00:00<00:00, 14665.40it/s]
